In [15]:
import os
import cv2
# import uuid
from functions import *
from ultralytics import YOLO
from paddleocr import PaddleOCR
from collections import defaultdict

In [ ]:
weight_vehicle_detection_path = "D:/WorkSpace/Vehicle_tracking/weights/vehicle_detection.pt"
tracker_path = "D:/WorkSpace/Vehicle_tracking/VehicleDetection/src/test/tracker/botsort.yaml"
weight_license_plate_detection_path = "D:/WorkSpace/Vehicle_tracking/weights/license_plate_detection.pt"

video_path = "D:/WorkSpace/Vehicle_tracking/LicenseOCR/data/IMG_4525.MOV"
device = "cuda"
class_names = ["car", "motor", "bus", "truck"]
colors = [
    (0, 0, 255),
    (0, 255, 0),
    (255, 0, 0),
    (0, 255, 255) 
]

model_vehicle = YOLO(weight_vehicle_detection_path)
model_vehicle.eval()
model_license_plate = YOLO(weight_license_plate_detection_path)
model_license_plate.eval()
ocr_model = PaddleOCR(
    use_doc_orientation_classify=False,
    use_doc_unwarping=False,
    use_textline_orientation=False
)

Creating model: ('PP-OCRv5_server_det', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\Window10\.paddlex\official_models\PP-OCRv5_server_det`.
Creating model: ('PP-OCRv5_server_rec', None)
Model files already exist. Using cached files. To redownload, please delete the directory manually: `C:\Users\Window10\.paddlex\official_models\PP-OCRv5_server_rec`.


In [ ]:
cap = cv2.VideoCapture(video_path)
fps = cap.get(cv2.CAP_PROP_FPS)
w = cap.get(cv2.CAP_PROP_FRAME_WIDTH)
h = cap.get(cv2.CAP_PROP_FRAME_HEIGHT)
output_filename = 'output.mp4'

fourcc = cv2.VideoWriter_fourcc(*'mp4v') 
scale = 0.4
out = cv2.VideoWriter(output_filename, fourcc, fps, (int(w*scale), int(h*scale)))
skip = 0.2
skip_frame = max(1, int(fps*skip))
frame_count = 0
scale_plot = 0.4
scale_process = 0.4

In [ ]:
# create_data = SelectAreaDetect(video_path, scale_plot)
# create_data.create_txt()

In [ ]:
CONFUSIONS_TO_NUMBER = {
    'B': '8',
    'G': '6',
    'Z': '2',
    'S': '5',
    'O': '0',
    'I': '1',
    'D': '0'
}

CONFUSIONS_TO_LETTER = {
    '8': 'B',
    '6': 'G',
    '9': 'B',
    '2': 'Z',
    '5': 'S',
    '0': 'O',
    '1': 'I',
    '3': 'B',
    '7': 'Z'
}

def correct_char(char, must_be_letter=False):
    if must_be_letter:
        return CONFUSIONS_TO_LETTER.get(char, char)
    else:
        return CONFUSIONS_TO_NUMBER.get(char, char)

def post_process_license_plate(raw_text):
    clean_text = re.sub(r'[^A-Z0-9]', '', str(raw_text).upper())
    corrected_chars = []
    template = "NNLNNNNNNNN"
    for i, char in enumerate(clean_text):
        must_be_letter = (template[i] == 'L')
        corrected_chars.append(correct_char(char, must_be_letter))

    return "".join(corrected_chars)

def plot_box(data, frame):
    for obj in data:
        x1, y1, x2, y2 = obj['bbox']
        id_ = obj['id']
        cl_name = class_names[int(obj['cls'])]
        color = obj['color']
        label = obj['label']
        if label == "":
            label = "Can't OCR"
        cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
        cv2.putText(frame, f"{label}", (x1, y2-3), cv2.FONT_HERSHEY_SIMPLEX, 0.7, color, 2, cv2.LINE_AA)
    return frame
        

In [ ]:
import torch
print(torch.version.cuda)
print(torch.cuda.is_available())


12.1
True


In [ ]:
# mask = gen_mask("IMG_4525/IMG_4525.txt", "IMG_4525/IMG_4525.png", scale_process)
# lass_detect = defaultdict()
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1
    if frame_count % skip_frame != 0:
        continue
    
    frame = cv2.resize(frame, (int(w*scale), int(h*scale)))
    # frame, frame_pr = process_image(frame, mask, scale_process)
    vehicle_result = model_vehicle.track(
        frame, 
        conf=0.75, 
        iou=0.7,
        tracker=tracker_path,
        device=device,
        persist=True,
        verbose=False 
    )[0]
    vehicle_data = []
    for bbox in vehicle_result.boxes:
        if bbox.id is not None:
            x1, y1, x2, y2 = bbox.xyxy[0].cpu().numpy().astype(int)
            id_ = bbox.id[0].cpu().numpy().astype(int)
            cls = bbox.cls[0].cpu().numpy().astype(int)
            color = colors[cls]
            obj = frame[y1:y2, x1:x2]
            try:
                lp_result = model_license_plate.predict(obj, conf=0.8, verbose=False, iou=0.8, device=device)[0].boxes[0]
                lx1, ly1, lx2, ly2 = lp_result.xyxy[0].cpu().numpy().astype(int)
                license = obj[ly1:ly2, lx1:lx2]
                ocr_result = ocr_model.predict(license)[0]
                label = ""
                # conf=0
                if ocr_result and ocr_result['rec_texts']:
                    num_texts = len(ocr_result['rec_texts'])
                    if num_texts > 0:
                        for j in range(num_texts):
                            label += ocr_result['rec_texts'][j]
                            # conf += ocr_result['rec_scores'][j]
                    # conf /= num_texts
                    label = post_process_license_plate(label)
                    # if id_ not in lass_detect:
                    #     lass_detect[id_] = {
                    #         'label': label,
                    #         'conf': conf
                    #     }
                    # else:
                    #     if conf > lass_detect[id_]['conf']:
                    #         lass_detect[id_] = {
                    #             'label': label,
                    #             'conf': conf
                    #         }
                    #     else:
                    #         label = lass_detect[id_]['label']
                    #         conf = lass_detect[id_]['conf']
                            
                vehicle_data.append({
                    'id': int(id_),
                    'cls': cls,
                    'color': color,
                    'bbox': (x1, y1, x2, y2),
                    'label': label
                })
            except:
                pass
    frame = plot_box(vehicle_data, frame)
    # cv2.imshow("Live", frame)
    out.write(frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
cap.release()
out.release()
cv2.destroyAllWindows()